In [1]:
%matplotlib widget
from matplotlib import pyplot as plt
import numpy as np
import mlc

# import dependencies for PCA
#from sklearn.decomposition import PCA
#from sklearn.manifold import TSNE

#from mlc.data.mnist.dataset import MNIST
#from mlc.util.model import load_checkpoint

In [4]:
import argparse
from mlc.model.cnn_gan.model import CNNGAN

# Create parser and add arguments
parser = argparse.ArgumentParser()
CNNGAN.add_arguments(parser)

# Parse empty args to use all defaults
args = parser.parse_args([])

# Convert Namespace to dict
args_dict = vars(args)

# Instantiate model with args
model = CNNGAN(args_dict)
model

CNNGAN: Image channels: 3, Image dim: 64, Latent dim: 128, Layer channels: 24, Use batchnorm: both


CNNGAN(
  (generator): Sequential(
    (0): Linear(in_features=128, out_features=384, bias=False)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Unflatten(dim=1, unflattened_size=(24, 4, 4))
    (3): ConvTranspose2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1), bias=False)
    (4): BatchNorm1d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm1d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.01)
    (9): ConvTranspose2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1), bias=False)
    (10): BatchNorm1d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.01)
    (12): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm1d(24, eps

In [12]:
x = np.random.randn(2, 128).astype(np.float32)
x = torch.from_numpy(x)
for layer in model.generator:
    x = layer(x)
    print(f"{layer.__class__.__name__} output shape: {x.shape}")

Linear output shape: torch.Size([2, 384])
LeakyReLU output shape: torch.Size([2, 384])
Unflatten output shape: torch.Size([2, 24, 4, 4])
ConvTranspose2d output shape: torch.Size([2, 24, 8, 8])


ValueError: expected 2D or 3D input (got 4D input)

In [ ]:
# get validation data
dataset = MNIST({"noise": 0, "normalize": True})
validation_data = dataset.get_fold("train")

In [ ]:
fig, ax = plt.subplots(1, 10, figsize=(10, 2), sharey=True)
# show some data
for i in range(10):
    data, _ = validation_data[i]
    with torch.no_grad():
        data = data.unsqueeze(0)
    ax[i].imshow(data.squeeze().cpu().numpy(), cmap="gray")
    ax[i].axis("off")

In [ ]:
G = model.generator.to("cuda")
G = G.eval()

In [ ]:
Z = torch.randn(100, model.latent_dimension()).to("cuda") * 0.1
# normalize Z
Z = (Z - torch.tensor([0.5]).to("cuda")) / torch.tensor([0.5]).to("cuda")
X = G(Z).cpu().detach().numpy()
# show generated data
fig, ax = plt.subplots(10, 10, figsize=(10, 10), sharey=True)
for i in range(10):
    for j in range(10):
        ax[i, j].imshow(X[i * 10 + j].reshape(28, 28), cmap="gray")
        ax[i, j].axis("off")
plt.show()

In [ ]:
z00, z01 = Z[27], Z[28]
# interpolate between two latent vectors
Z_interp = []
for alpha in np.linspace(0, 1, 10):
    z = (1 - alpha) * z00 + alpha * z01
    Z_interp.append(z)

# show interpolated data
X_interp = G(torch.stack(Z_interp)).cpu().detach().numpy()
fig, ax = plt.subplots(1, 10, figsize=(10, 2), sharey=True)
for i in range(10):
    ax[i].imshow(X_interp[i].reshape(28, 28), cmap="gray")
    ax[i].axis("off")
plt.show()

In [ ]:
z00, z01 = Z[27], Z[28]
z10, z11 = Z[70], Z[56]

# bilinear interpolation between two latent vectors
Z_interp = []
for alpha in np.linspace(0, 1, 10):
    for beta in np.linspace(0, 1, 10):
        z = (1 - alpha) * (1 - beta) * z00 + (1 - alpha) * beta * z01 + alpha * (1 - beta) * z10 + alpha * beta * z11
        Z_interp.append(z)
# show interpolated data
X_interp = G(torch.stack(Z_interp)).cpu().detach().numpy()
fig, ax = plt.subplots(10, 10, figsize=(10, 10), sharey=True)
for i in range(10):
    for j in range(10):
        ax[i, j].imshow(X_interp[10 * i + j].reshape(28, 28), cmap="gray")
        ax[i, j].axis("off")
plt.show()

In [ ]:
# 7 - 1 + 4
z1 = Z[19]
z2 = Z[49]
z3 = Z[98]

# apply model to the latent vectors
X1 = G(z1.unsqueeze(0)).cpu().detach().numpy()
X2 = G(z2.unsqueeze(0)).cpu().detach().numpy()
X3 = G(z3.unsqueeze(0)).cpu().detach().numpy()
# show the generated data
fig, ax = plt.subplots(1, 3, figsize=(6, 2), sharey=True)
ax[0].imshow(X1.reshape(28, 28), cmap="gray")
ax[0].axis("off")
ax[1].imshow(X2.reshape(28, 28), cmap="gray")
ax[1].axis("off")
ax[2].imshow(X3.reshape(28, 28), cmap="gray")
ax[2].axis("off")
plt.show()

z_r = z1 - z2 + z3
# apply model to the resulting latent vector
X_r = G(z_r.unsqueeze(0)).cpu().detach().numpy()
# show the generated data
fig, ax = plt.subplots(1, 1, figsize=(2, 2), sharey=True)
ax.imshow(X_r.reshape(28, 28), cmap="gray")
ax.axis("off")
plt.show()